# Krigeage sur une Variable des échantillons de mousse

Dans ce notebok, on se concentre uniquement sur le krigeage à une variable et uniquement sur les variables correspondant aux concentrations en métaux dans les mousses.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import gstlearn as gl
import gstlearn.plot as gp
from bramm_data_analysis import loaders
import matplotlib.pyplot as plt
from bramm_data_analysis.spatial import RegularGrid
from bramm_data_analysis.loaders.preprocessing import QuantileThreshold

## Données

### Chargement

On charge les données pour une variable, ici le cuivre. Les données de mousse correspondent aux données des 3 feuilles de données, fusionnées sur le code de site et sur le code d'échantillon pour former un grand jeu de données. Les données sont ensuite filtrées pour ne conserver que la variable interessante (ici le cuivre).

La gestion des valeurs dupliquées se fait en deux étapes. Dans un premier temps, on aggrège les données correspondants à un même échantillon (les données ayant même date, même longitude et même latitude), en faisant leur moyenne. Ensuite, si il reste des données ayant la même position spatial (il n'est plus question de date ici), on conserve les données les plus récentes.

On s'assure ensuite de l'absence de `NaN` dans les données et on convertit le jeu de données (on conservant seulement la longitude, la latitute et les valeurs de cuivre) au format `Db` de `gstlearn` pour effectuer un premier krigeage.

Pour plus d'informations sur le chargement des données, se référer [aux fonctions de chargement de données](../src/bramm_data_analysis/loaders/core.py).

On retire également les 5% des valeurs les plus élevées, pour se débarrasser d'éventuelles erreurs de mesure causant des valeurs trop élevées. De même on retire les 5% des valeurs les plus basses.

In [ ]:
x1 = "longitude"
x2 = "latitude"
z = "copper"

# Moss
moss_data_path = Path("../data/Mines_2024.xlsx")
observations = loaders.from_moss_csv(moss_data_path).retrieve_db(
    xs=["longitude", "latitude"],
    zs=z,
    duplicates_handling_strategy="mean",
    thresholds=[QuantileThreshold(field=z, lower=0.05, upper=0.95)],
)
observations.plot(z, flagLegendColor=True)

### Définition d'une grille pour le krigeage

On définit une grille régulière, de maille `step` à l'aide de `gstlearn.Dbgrid.create`. Ensuite, à l'aide de la library `geopandas` et du tracé de la [france métropolitaine](../data/metropole.json) (obtenu sur Github [ici](https://github.com/gregoiredavid/france-geojson/blob/master/metropole.geojson)), on créé un selecteur dans la grid régulière pour indiquer les points à l'intérieur de la france métropolitaine et de la corse. Pour plus de détailler, se référer au module [`spatial`](../src/bramm_data_analysis/spatial/__init__.py) ou au notebook [`spatial_division`](../notebooks/spatial_division.ipynb).

In [ ]:
step = 0.1
grid = RegularGrid.from_boundary_path(Path("../data/metropole.json"))
db_grid = grid.retrieve_grid(step)
db_grid.display()

## Etude Statistique

### Variogramme

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(
    ndir=2, npas=10, dpas=0.5, toldis=0.2
)
vario2dir = gl.Vario(varioParamMulti)
err = vario2dir.compute(observations)
ax = gp.varmod(vario2dir)

In [ ]:
fitmod = gl.Model()
types = [gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN]
err = fitmod.fit(vario2dir, types=types)
ax = gp.varmod(vario2dir, fitmod)

### Krigeage avec un unique voisinage

In [ ]:
neigh = gl.NeighUnique.create()

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmod,
    neigh=neigh,
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("SK"),
)
# visualization
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"SK.{z}.estim", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(title="Unique Neigh Kriging over whole Grid")
plt.show()
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"SK.{z}.stdev", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(title="Unique Neigh Kriging std-dev over whole Grid")
plt.show()

In [ ]:
# Cross-Validation

err = gl.xvalid(
    db=observations,
    model=fitmod,
    neigh=neigh,
    flag_xvalid_est=1,
    flag_xvalid_std=1,
    namconv=gl.NamingConvention.create("CV_unique", flag_locator=False),
)
fig, ax = gp.initGeographic()
ax.raster(db_grid, name="inland")
ax.symbol(
    observations,
    nameSize=f"CV_unique.{z}.esterr",
    flagAbsSize=True,
    flagLegendSize=True,
)
ax.decoration(title="Unique Neigh Cross-Validation scores (abs. value)")
plt.show()

### Krigeage avec un voisinage mobile 

In [ ]:
neigh = gl.NeighMoving.create(nmini=1, nmaxi=1)

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=db_grid,
    model=fitmod,
    neigh=neigh,
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("Move_SK"),
)

In [ ]:
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"Move_SK.{z}.estim", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(
    title="Unique Neigh Kriging over whole Grid",
)
plt.show()
fig, ax = gp.initGeographic()
ax.raster(db_grid, name=f"Move_SK.{z}.stdev", flagLegend=True)
ax.symbol(observations, c="black")
ax.decoration(title="Moving Neigh Kriging std-dev over whole Grid")
plt.show()

In [ ]:
# Cross-Validation

err = gl.xvalid(
    db=observations,
    model=fitmod,
    neigh=neigh,
    flag_xvalid_est=1,
    flag_xvalid_std=1,
    namconv=gl.NamingConvention.create("CV_moving", flag_locator=False),
)
fig, ax = gp.initGeographic()
ax.raster(db_grid, name="inland")
ax.symbol(
    observations,
    nameSize=f"CV_moving.{z}.esterr",
    flagAbsSize=True,
    flagLegendSize=True,
)
ax.decoration(title="Moving Neigh Cross-Validation scores (abs. value)")
plt.show()

La validation croisée donne finalement de meilleurs scores pour le krigeage effectué avec un unique voisinage plutôt qu'avec un voisinage mobile. 

Pour explorer plus en détail le jeu de données, n'héstez pas à rejouer ce notebook avec différentes variables, par exemple :

```python
z = "iron"
```

```python
z = "aluminium"
```

``` python
z = "calcium"
```